# Dimensionality Reduction II

Guarda cosa succede se fai PCA, NMF o ICA: ma se volessi fare dimensionality reduction in un altro modo..?

`RandomForestRegressor` è un metodo di regressione, ma vediamo cosa fa: quali sono i pixel "più importanti"?

---

### Non-Linear Dimensionality Reduction

Uso i concetti di *manifold learning* e *nonlinear dimensionality reduction* per "imparare" qualcosa dai dati. -> `sklearn.manifold` è il pacchetto da usare.

Prendi il manifold "a forma di S" che ti fa vedere e applica PCA: esce uno schifo, non separi niente. Applichi IsoMap o LLE? Funziona molto meglio!

### LLE: Locally Linear Embedding

LLE attempts to embed high-D data in a lower-D space. L'aspetto *cruciale* del metodo è che cerca di **preservare la geometria** dei "local neighborhoods" attorno a ciascun punto. Nel caso del disegno "ad S" di prima, cerca di "stendere" il foglio di punti.

**Step 1**: definire la local geometry

* local neighborhood di P = { i $k$ punti più vicini al punto P }
* per ogni punto P calcolo i pesi e ricostruisco un punto dai suoi $k$ nearest neighbors con $\mathcal{E}_1(W) = \left|X - WX\right|^2$, dove X è una matrice NxK e W è una matrice NxN che minimizza l'errore di ricostruzione.

In pratica, sto trovando un **iperpiano** che descrive la **local surface @ each point** within the data set (?)(?)(?)

##### NHC CODE

**Step 2**: embed within a lower dimensional space

* W becomes very sparse for $k \ll N $
* minimize :$ \mathcal{E}_2(Y) = \left|Y - W Y\right|^2$,
with $W$ fixed to find an $N$ by $d$ matrix ($d$ is the new dimensionality).

**Step 3**: trovo una dimensionalità minore in cui infilo i dati, Y, che rispecchia la geometria dei dati nella matrice dei dati originale, X

> Lo step 1 richiede una nearest-neighbor search. <br> Lo step 2 richiede una *eigenvalue decomposition* di una matrice.

### IsoMap: Isometric Mapping

IsoMap cerca di trovare una riduzione di dimensionalità della data matrix che **mantiene la geodesic distance tra i punti invariata**! NON USA la distanza Euclidea, ma la distanza della geodetica!

For the final step of finding the optimal low-dimensional embedding, MDS algorithm technqiues are used to minimize the reconstruction error:

$$ \mathcal{E}_{XY} = |\tau(D_X) - \tau(D_Y)|^2 $$

between the original data set $X$ and the low-dimensional embedding $Y$, where $\tau$ is some operator on the distance matrices.

### t-SNE: t-distributed Stochastic Neighbor Embedding

**NON È TRATTATO NEL LIBRO** ma c'è in sklearn, ed è molto potente. È un manifold-learning algorithm, nuovissimo. 

PCA vuole massimizzare la varianza, e cerca di tenere lontani i punti più diversi. t-SNE vuole preservare la distanza tra i punti più vicini, per tenere vicini i punti più simili.

A high-level description of the steps in t-SNE are as follows:

**STEP 1**: *measure the similarity between points in the original high-dimensional space*
- this is done by centering a multivariate Gaussian on each point and measuring the pdf value of all neighboors.
- the bandwidth of the Gaussian can be chosen to tweak something called the *perplexity*, which essentially relates to the number of nearest neighboors being considered.
- the result is a matrix of probabilities of each point under a Gaussian centered under every other point, $p_{ij}$.
- this "similarity matrix" is a measure of local structure.

**STEP 2**: *find a lower-dimensional mapping that preserves the similarities, $p_{ij}$*
- a heavy-tailed Student's $t$-distribution with $1$-degree of freedom (i.e. a Cauchy distribution) is used to model the probabilities between pairs of points in the low-dimensional embedding, $q_{ij}$.
- this allows dissimilar objects to be modeled as being far apart on the map. 
- A quantity known as the [Kullback-Leibler divergence](https://www.wikiwand.com/en/Kullback%E2%80%93Leibler_divergence) is used to minimize differences between $p_{ij}$ and $q_{ij}$. 

Try it on the digits data. You'll need to import `TSNE` from `sklearn.manifold`, instantiate it with 2 components, then do a `fit_transform` on the original data.

##### GUARDA IL NOTEBOOK PERCHÉ È ECCEZIONALE!

### Drawbacks of non-linear embeddings tecniques

- **Noisy and gappy data**. For our S problem, imagine a point at x=0,y=0 but unconstrained in z. The algorithm cannot possibly know where to put it on the S. Manifold learning tecniques are intrisically bad for missing data. 
- **Tuning parameters**. Sophisticated things needs to be tuned (I didn't even mention the learning rate parmeter in t-SNE!)
- **Dimensionality**. In PCA, the eigenvalue give you a sense of the intrinsic dimensionality, i.e. how many dimensions you should keep. This is not the case with non-linear approaches.
- **Outliers** can be a big problem for these tools. There's a nice example on this in Fig 7.9 in the textbook. If you look at the code, the first have to filter out outliers with a different algorithm before applying LLE successfully.
- **Reconstruction**. Non-linear mapping does not provide a set of basis function like PCA, making data reconstruction  less clean (need to use e.g. nearest neighbors).

## Better Dataviz

We often use dimensionality reduction to help with data visualization, so it seems appropriate to briefly talk about some different ways to improve our data visualization using plotting tools. 

This is an illustration of the power of [Bokeh](https://bokeh.org/), a language to create interacte browser-based dashboards for data visualization. The following cell will create and launch a new browser tab (chrome for me) that shows a set of interactive scatter plots for the penguin data set. This uses the data-visualization principles of [**brushing & linking**](https://www.wikiwand.com/en/Brushing_and_linking).

- ***Linking*** refers to having a connected series of data representations (e.g. histograms or scatter plots), where a change in one parameter or a subset of parameters in one representation is then reflected in the others.
- ***Brushing*** refers to highlighting subsets of data within one representation such that this subset is also highlighted in its other representations through the linked view.

In [86]:
import seaborn as sns
penguins = sns.load_dataset("penguins")
from bokeh.plotting import *
from bokeh.models import ColumnDataSource

# prepare some data
N = 300
x0 = penguins['bill_depth_mm']
x1 = penguins['flipper_length_mm']
y = penguins['body_mass_g']
species = penguins['species']

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
source = ColumnDataSource(data=dict(x0=x0, x1=x1, 
                                    y=y, species=species))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("species", "@species"),
]

# create a new plot and add a renderer
left = figure(tools=TOOLS, tooltips=TOOLTIPS, 
              width=350, height=350, title=None,
              x_axis_label ="bill_depth_mm",
              y_axis_label ="body_mass_g")
left.circle('x0', 'y', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, tooltips=TOOLTIPS, 
               width=350, height=350, title=None,
              x_axis_label ="flipper_length_mm",
               y_axis_label ="body_mass_g")
right.circle('x1', 'y', source=source)

# put the subplots in a gridplot
p = gridplot([[left, right]])

# show the results
show(p,browser="firefox")

### Excercise for home
Implement a bokeh visualization in our of the excercises we've done so far (for instance the GRB one, or the digits above) and see if you find something new

## Density Estimation <a class="anchor" id="one"></a>

Inferring the pdf of a sample of data is known as ***density estimation***. Essentially we are smoothing the data to correct for the finiteness of our sample and to better recover the underlying distribution.

We have seen some hints in a previous lectures where we discussed histogram bins and KDEs. Let's recap and expand here.


Density estimation is useful because:
- identifying low probability regions can help uncover rare sources. 
- if the data can be divided into sub-samples, one can estimate the pdf for each subsample and, in turn determine classifications for new objects.

### Non-parametric Density Estimation <a class="anchor" id="onea"></a>

*Nonparametric* density estimation is useful when we know nothing about the underlying distribution of the data, since we don't have to specify a functional form. This flexibility allows us to capture the shape of the distribution well, at the expense of more difficulty interpreting the results.

#### Kernel Density Estimation (KDE)

[*Kernel Density Estimation (KDE)*](https://en.wikipedia.org/wiki/Kernel_density_estimation) is the standard approach for non-parametric density estimation.

Guarda il notebook: fa lo stesso numero di bin sul dataset, ma coi bin sfasati.***The choice of number of bins and the location of bin centers can really change the histogram that we make.***

The next panels are what happens if we center the bins on each point. This is an example of **kernel density estimation** using a "***top-hat***" kernel. It is a good description of the data, but pretty ugly.

We can make it look nicer by choosing a different kernel, i.e. a different bin shape. The next plot shows a **KDE using a Gaussian kernel**. ( è lo stesso lavoro di Nati visto nella lezione del 02/mag )

We can think of KDE as replacing the points with "clouds". Each cloud is described by the kernel $K(u)$, where $K(u)$ can be any function that is:
- smooth,
- postive definite,
- normalizes to unity, 
- has zero mean,
- has positive variance. 

A common kernel is the **Gaussian kernel** that we just used above:

$$ K(u) = \frac{1}{(2\pi)^{D/2}}\exp{(-u^2/2)}$$

where $D$ denotes the dimensionality of the data. Once a kernel is chosen the KDE at a point, $x$, is given by 

$$ \hat{f}(x) = \frac{1}{Nh^D}\sum_{i=1}^N K\left(\frac{d(x,x_i)}{h}\right),$$

where $\hat{f}$ is an ***estimator*** of our distribution.

The argument of $K$ is just some measure of the distance between $x$ and each $x_i$. Normally $d(x,x_i) = (x-x_i)$. For the gaussian kernel that makes $h=\sigma$. So, $h$ represents the "width" or what is usually called the **"bandwidth"** in this context.

**The Epanechnikov kernel is "optimal" because it minimizes the variance of the kernel density estimate**: 

$$K(x) = \frac{3}{4}(1-x^2),$$

for $|x|\le 1$ and 0 otherwise.

#### How do we determine the optimal kernel bandwidth?

You know the answer to this one already: cross validation. Go back to lecture 14.



#### 2-D Histograms

Vedi notebook per il sample code. What we are doing here is using KDE to set the plot color to indicate the relative density of the points.  This is essentially a 2-D histogram.

We now want do a data-drive selection of the bandwidth. That's a cross validation on a 2d KDE, which sounds sounds complicated. Actually, look at how easy this is! Just copy the example from above to a new cell and splice in the cross validation code to produce a new plot with the "optimal" bandwidth. Basically, splice in the lines of code for `GridSearchCV` between the lines setting `X` and instantiating `kde`.  Then replace the hardcoded bandwidth with the optimal value you computed.

That's it really, and it just works. This is a good example on how modern python libraries can really make you data-science life so much easier!

---

#### Nearest-Neighbor Density Estimation

Another very simple way to estimate the density of an $N$-dimensional distribution is to look to the nearest object (or the $K$ nearest objects) and compute their distances, $d_K$.  This is the [$K$-Nearest Neighbor](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) algorithm. The density at a given point, $x$ is estimated as

$$\hat{f}_K(x) = \frac{K}{V_D(d_K)},$$

where $V_D(d)$ is given generically by $\frac{2d^D\pi^{D/2}}{D\Gamma(D/2)}$ where $\Gamma$ is the complete gamma function, and this formula reduces to the usual equations for area and volume in 2 and 3 dimensions, respectively.

We can simplify this to 

$$\hat{f}_K(x) = \frac{C}{d_K^D}$$

since the constant, $C$ can be evaluated at the end.

This estimator has some intrinsic bias, which can be reduced by considering *all* $K$ nearest neighbors:
$$\hat{f}_K(x) = \frac{C}{\sum_{i=1}^K d_i^D}$$

See the [Scikit-Learn `neighbors` documentation](http://scikit-learn.org/stable/modules/neighbors.html) for more information.

Ivezic, Figure 6.5 compares a Nearest Neighbor ($k=10$) with a KDE algorithm. Let's what happens as you increase the number of neighbors used.

### Parametric Density Estimation <a class="anchor" id="oneb"></a>

#### Gaussian Mixture Models (GMM)

We've done an excercise on this already, so let me recap this quickly for completeness. KDE centers each bin (or rather, kernel) at each point.  In a [**mixture model**](https://en.wikipedia.org/wiki/Mixture_model) we don't use a kernel for each data point, but rather we fit for the ***locations of the kernels*** in addition to the width. So a mixture model is sort of a hybrid between a traditional (fixed bin location/size) histogram and KDE. 

- Using lots of kernels (maybe even more than the AIC or BIC score suggests) may make sense if you just want to provide an accurate description of the data (as in density estimation).  
- Using fewer kernels makes mixture models more like clustering (later today), where the suggestion is still to use many kernels in order to divide the sample into real clusters and "background".

Gaussians are the most commonly used components for mixture models.  So, the pdf is modeled by a sum of Gaussians:

$$p(x) = \sum_{k=1}^N \alpha_k \mathscr{N}(x|\mu_k,\Sigma_k),$$

where $\alpha_k$ are the "mixing coefficients" with $0\le \alpha_k \le 1$ and $\sum_{k=1}^N \alpha_k = 1$.

We can solve for the parameters using maximum likelihood analyis as we have discussed previously.
However, this can be complicated in multiple dimensions, requiring the use of [**Expectation Maximization (EM)**](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) methods (see textbook for details).

Ivezic Figure 4.2 (next cell) provides an example in 1-D.

### Datasets you can play with

If irises, penguins, or hand-written digits aren't your thing, then have a look through some of these public data repositories. Explore!

- [https://github.com/caesar0301/awesome-public-datasets?utm_content=buffer4245d&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer](https://github.com/caesar0301/awesome-public-datasets?utm_content=buffer4245d&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer)
- [http://www.datasciencecentral.com/m/blogpost?id=6448529%3ABlogPost%3A318739](http://www.datasciencecentral.com/m/blogpost?id=6448529%3ABlogPost%3A318739)
- [http://www.kdnuggets.com/2015/04/awesome-public-datasets-github.html](http://www.kdnuggets.com/2015/04/awesome-public-datasets-github.html)

# And now... Pills of modern research with two professional astrostatisticians!

For today I've invited two of my scientific collaborators to give you some examples on how current astrophysical problems are tackled not just using, but also *developing* new statistical tools:  

- Matthew Mould (University of Birmingham, UK). [Slides](https://github.com/dgerosa/astrostatistics_bicocca_2022/blob/main/lectures/T01_Mould.pdf)
- Riccardo Buscicchio (University of Milano-Bicocca, Italy) [Slides](https://github.com/dgerosa/astrostatistics_bicocca_2022/blob/main/lectures/T02_Buscicchio.pdf)